In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

# URL de la page principale des articles
base_url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"

# Fonction pour scraper les articles de la dernière semaine
def scrape_ecofin():
    articles = []
    today = datetime.today()
    one_week_ago = today - timedelta(days=7)

    # Récupérer la page principale
    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Trouver tous les articles
    for article in soup.find_all("article"):  # Ajuste selon la structure du site
        title = article.find("h2").text.strip()
        link = article.find("a")["href"]
        date_text = article.find("time")["datetime"]  # Date en format ISO
        article_date = datetime.fromisoformat(date_text)

        # Filtrer sur les 7 derniers jours
        if article_date >= one_week_ago:
            articles.append({"title": title, "link": link, "date": article_date})

    return pd.DataFrame(articles)

# Sauvegarder les articles dans un fichier CSV
articles_df = scrape_ecofin()
articles_df.to_csv("articles_ecofin.csv", index=False)
print("Articles sauvegardés :", articles_df)


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

# URL de la page principale
base_url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"

# Fonction pour scraper les articles d'une seule page
def scrape_page(url, one_week_ago):
    articles = []
    
    # Récupérer le contenu HTML de la page
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Localiser la table contenant les articles
    table = soup.find("table", class_="ts")  # Table avec les articles
    
    if table:
        for row in table.find_all("td", class_="top news-btn"):
            # Extraire le lien, le titre, et la date
            link_tag = row.find("a")
            title = link_tag.text.strip() if link_tag else "Titre indisponible"
            relative_link = link_tag["href"] if link_tag else None
            link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None
            
            date_span = row.find_next("span", class_="if nsa")
            date_text = date_span.text.strip() if date_span else None
            
            # Convertir la date au format datetime
            if date_text:
                try:
                    article_date = datetime.strptime(date_text, "%d/%m/%Y")
                except ValueError:
                    article_date = None
                
                # Filtrer les articles des 7 derniers jours
                if article_date and article_date >= one_week_ago:
                    articles.append({
                        "title": title,
                        "link": link,
                        "date": article_date.strftime("%Y-%m-%d")
                    })
    
    return articles

# Fonction pour gérer la pagination
def scrape_all_pages():
    articles = []
    today = datetime.today()
    one_week_ago = today - timedelta(days=7)
    
    next_page_url = base_url
    while next_page_url:
        print(f"Scraping page : {next_page_url}")
        articles.extend(scrape_page(next_page_url, one_week_ago))
        
        # Trouver le lien vers la page suivante
        response = requests.get(next_page_url)
        soup = BeautifulSoup(response.content, "html.parser")
        next_page_tag = soup.find("a", text="Suivant")  # Lien vers la page suivante
        
        if next_page_tag:
            relative_next_page = next_page_tag["href"]
            next_page_url = f"https://www.agenceecofin.com{relative_next_page}"
        else:
            next_page_url = None  # Pas de page suivante
    
    return pd.DataFrame(articles)

# Scraper tous les articles et sauvegarder dans un CSV
articles_df = scrape_all_pages()
articles_df.to_csv("articles_ecofin.csv", index=False)
print("Articles sauvegardés :", articles_df)


Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results


/tmp/ipykernel_15951/2809485671.py:62: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  next_page_tag = soup.find("a", text="Suivant")  # Lien vers la page suivante


Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=7
Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=14
Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=21
Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=28
Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench

KeyboardInterrupt: 

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

# URL de la page principale
base_url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"


# Fonction pour scraper les articles d'une seule page
def scrape_page(url, one_week_ago):
    articles = []
    
    # Récupérer le contenu HTML de la page
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Localiser la table contenant les articles
    table = soup.find("table", class_="ts")  # Table avec les articles
    
    if table:
        for row in table.find_all("td", class_="top news-btn"):
            # Extraire le lien, le titre, et la date
            link_tag = row.find("a")
            title = link_tag.text.strip() if link_tag else "Titre indisponible"
            relative_link = link_tag["href"] if link_tag else None
            link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None
            
            date_span = row.find_next("span", class_="if nsa")
            date_text = date_span.text.strip() if date_span else None
            
            # Convertir la date au format datetime
            if date_text:
                try:
                    article_date = datetime.strptime(date_text, "%d/%m/%Y")
                except ValueError:
                    article_date = None
                
                # Filtrer les articles des 7 derniers jours
                if article_date and article_date >= one_week_ago:
                    articles.append({
                        "title": title,
                        "link": link,
                        "date": article_date.strftime("%Y-%m-%d")
                    })
    
    return articles

# Fonction pour récupérer le contenu d'un article
def scrape_article_content(article_url):
    response = requests.get(article_url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Trouver le contenu de l'article
    content_div = soup.find("div", class_="itemIntroText")
    if content_div:
        paragraphs = content_div.find_all("p", class_="texte textearticle")
        content = " ".join(p.text.strip() for p in paragraphs)
        return content
    return "Contenu indisponible"

# Fonction pour gérer la pagination
def scrape_all_pages():
    articles = []
    today = datetime.today()
    one_week_ago = today - timedelta(days=7)
    
    next_page_url = base_url
    while next_page_url:
        print(f"Scraping page : {next_page_url}")
        articles.extend(scrape_page(next_page_url, one_week_ago))
        
        # Trouver le lien vers la page suivante
        response = requests.get(next_page_url)
        soup = BeautifulSoup(response.content, "html.parser")
        next_page_tag = soup.find("a", text="Suivant")  # Lien vers la page suivante
        
        if next_page_tag:
            relative_next_page = next_page_tag["href"]
            next_page_url = f"https://www.agenceecofin.com{relative_next_page}"
        else:
            next_page_url = None  # Pas de page suivante
    
    # Récupérer le contenu de chaque article
    for article in articles:
        print(f"Scraping article content: {article['link']}")
        article["content"] = scrape_article_content(article["link"])
    
    return pd.DataFrame(articles)

# Scraper tous les articles et sauvegarder dans un CSV
articles_df = scrape_all_pages()
articles_df.to_csv("articles_ecofin.csv", index=False)
print("Articles sauvegardés :", articles_df)


Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results


/tmp/ipykernel_15951/1850910078.py:76: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  next_page_tag = soup.find("a", text="Suivant")  # Lien vers la page suivante


Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=7
Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=14
Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=21
Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=28
Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench

KeyboardInterrupt: 

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

# URL de la page principale
base_url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"


# Fonction pour scraper les articles d'une seule page
def scrape_page(url, one_week_ago):
    articles = []
    
    # Récupérer le contenu HTML de la page
    try:
        response = requests.get(url)
        response.raise_for_status()  # Vérifie les erreurs de requête
        soup = BeautifulSoup(response.content, "html.parser")
    except requests.RequestException as e:
        print(f"Erreur lors de la récupération de {url}: {e}")
        return articles
    
    # Localiser la table contenant les articles
    table = soup.find("table", class_="ts")  # Table avec les articles
    
    if table:
        for row in table.find_all("td", class_="top news-btn"):
            # Extraire le lien, le titre, et la date
            link_tag = row.find("a")
            title = link_tag.text.strip() if link_tag else "Titre indisponible"
            relative_link = link_tag["href"] if link_tag else None
            link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None
            
            date_span = row.find_next("span", class_="if nsa")
            date_text = date_span.text.strip() if date_span else None
            
            # Convertir la date au format datetime
            if date_text:
                try:
                    article_date = datetime.strptime(date_text, "%d/%m/%Y")
                except ValueError:
                    article_date = None
                
                # Filtrer les articles des 7 derniers jours
                if article_date and article_date >= one_week_ago:
                    articles.append({
                        "title": title,
                        "link": link,
                        "date": article_date.strftime("%Y-%m-%d"),
                        "date_obj": article_date  # Conserver l'objet date pour une utilisation ultérieure
                    })
    
    return articles

# Fonction pour récupérer le contenu d'un article
def scrape_article_content(article_url):
    try:
        response = requests.get(article_url)
        response.raise_for_status()  # Vérifie les erreurs de requête
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Trouver le contenu de l'article
        content_div = soup.find("div", class_="itemIntroText")
        if content_div:
            paragraphs = content_div.find_all("p", class_="texte textearticle")
            content = " ".join(p.text.strip() for p in paragraphs)
            return content
    except requests.RequestException as e:
        print(f"Erreur lors de la récupération de l'article {article_url}: {e}")
    
    return "Contenu indisponible"

# Fonction pour gérer la pagination
def scrape_all_pages():
    articles = []
    today = datetime.today()
    one_week_ago = today - timedelta(days=7)
    
    next_page_url = base_url
    while next_page_url:
        print(f"Scraping page : {next_page_url}")
        articles.extend(scrape_page(next_page_url, one_week_ago))
        
        # Trouver le lien vers la page suivante
        try:
            response = requests.get(next_page_url)
            response.raise_for_status()  # Vérifie les erreurs de requête
            soup = BeautifulSoup(response.content, "html.parser")
            next_page_tag = soup.find("a", text="Suivant")  # Lien vers la page suivante
            
            if next_page_tag:
                relative_next_page = next_page_tag["href"]
                next_page_url = f"https://www.agenceecofin.com{relative_next_page}"
            else:
                next_page_url = None  # Pas de page suivante
        except requests.RequestException as e:
            print(f"Erreur lors de la récupération de la page suivante: {e}")
            break  # Arrêtez-vous en cas d'erreur
    
    # Récupérer le contenu de chaque article
    for article in articles:
        if article["date_obj"] >= one_week_ago:  # Vérifier la date avant de scraper le contenu
            print(f"Scraping article content: {article['link']}")
            article["content"] = scrape_article_content(article["link"])
    
    return pd.DataFrame(articles)

# Scraper tous les articles et sauvegarder dans un CSV
articles_df = scrape_all_pages()
articles_df.to_csv("articles_ecofin.csv", index=False)
print("Articles sauvegardés :", articles_df)

Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results


/tmp/ipykernel_15951/2767026198.py:89: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  next_page_tag = soup.find("a", text="Suivant")  # Lien vers la page suivante


Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=7
Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=14
Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=21
Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=28
Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench

KeyboardInterrupt: 

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

# URL de la page principale
base_url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"

# Fonction pour scraper les articles d'une seule page
def scrape_page(url, one_week_ago):
    articles = []
    
    # Récupérer le contenu HTML de la page
    try:
        response = requests.get(url)
        response.raise_for_status()  # Vérifie les erreurs de requête
        soup = BeautifulSoup(response.content, "html.parser")
    except requests.RequestException as e:
        print(f"Erreur lors de la récupération de {url}: {e}")
        return articles
    
    # Localiser la table contenant les articles
    table = soup.find("table", class_="ts")  # Table avec les articles
    
    if table:
        for row in table.find_all("td", class_="top news-btn"):
            # Extraire le lien, le titre, et la date
            link_tag = row.find("a")
            title = link_tag.text.strip() if link_tag else "Titre indisponible"
            relative_link = link_tag["href"] if link_tag else None
            link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None
            
            date_span = row.find_next("span", class_="if nsa")
            date_text = date_span.text.strip() if date_span else None
            
            # Convertir la date au format datetime
            if date_text:
                try:
                    article_date = datetime.strptime(date_text, "%d/%m/%Y")
                except ValueError:
                    article_date = None
                
                # Filtrer les articles des 7 derniers jours
                if article_date and article_date >= one_week_ago:
                    articles.append({
                        "title": title,
                        "link": link,
                        "date": article_date.strftime("%Y-%m-%d"),
                        "date_obj": article_date  # Conserver l'objet date pour une utilisation ultérieure
                    })
    
    return articles

# Fonction pour récupérer le contenu d'un article
def scrape_article_content(article_url):
    try:
        response = requests.get(article_url)
        response.raise_for_status()  # Vérifie les erreurs de requête
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Trouver le contenu de l'article
        content_div = soup.find("div", class_="itemIntroText")
        if content_div:
            paragraphs = content_div.find_all("p", class_="texte textearticle")
            content = " ".join(p.text.strip() for p in paragraphs)
            return content
    except requests.RequestException as e:
        print(f"Erreur lors de la récupération de l'article {article_url}: {e}")
    
    return "Contenu indisponible"

# Fonction pour gérer la pagination
def scrape_all_pages():
    articles = []
    today = datetime.today()
    one_week_ago = today - timedelta(days=7)
    
    next_page_url = base_url
    while next_page_url:
        print(f"Scraping page : {next_page_url}")
        articles.extend(scrape_page(next_page_url, one_week_ago))
        
        # Trouver le lien vers la page suivante
        try:
            response = requests.get(next_page_url)
            response.raise_for_status()  # Vérifie les erreurs de requête
            soup = BeautifulSoup(response.content, "html.parser")
            next_page_tag = soup.find("a", text="Suivant")  # Lien vers la page suivante
            
            if next_page_tag:
                relative_next_page = next_page_tag["href"]
                next_page_url = f"https://www.agenceecofin.com{relative_next_page}"
            else:
                next_page_url = None  # Pas de page suivante
        except requests.RequestException as e:
            print(f"Erreur lors de la récupération de la page suivante: {e}")
            break  # Arrêtez-vous en cas d'erreur
    
    # Récupérer le contenu de chaque article
    for article in articles:
        if article["date_obj"] >= one_week_ago:  # Vérifier la date avant de scraper le contenu
            print(f"Scraping article content: {article['link']}")
            article["content"] = scrape_article_content(article["link"])
    
    return pd.DataFrame(articles)

# Scraper tous les articles et sauvegarder dans un CSV
articles_df = scrape_all_pages()
articles_df.to_csv("articles_ecofin.csv", index=False)
print("Articles sauvegardés :", articles_df)

Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results


/tmp/ipykernel_15951/820680180.py:88: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  next_page_tag = soup.find("a", text="Suivant")  # Lien vers la page suivante


Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=7
Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=14
Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=21
Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=28
Scraping page : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench

KeyboardInterrupt: 

In [7]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# URL de la page principale
url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"


# Fonction pour scraper les articles de la première page
def scrape_first_page(url):
    articles = []
    
    # Récupérer le contenu HTML de la page
    try:
        response = requests.get(url)
        response.raise_for_status()  # Vérifie les erreurs de requête
        soup = BeautifulSoup(response.content, "html.parser")
    except requests.RequestException as e:
        print(f"Erreur lors de la récupération de {url}: {e}")
        return articles
    
    # Localiser la table contenant les articles
    table = soup.find("table", class_="ts")  # Table avec les articles
    
    if table:
        rows = table.find_all("td", class_="top news-btn")[:7]  # Limiter à 7 articles
        for row in rows:
            # Extraire le lien, le titre, et la date
            link_tag = row.find("a")
            title = link_tag.text.strip() if link_tag else "Titre indisponible"
            relative_link = link_tag["href"] if link_tag else None
            link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None
            
            date_span = row.find_next("span", class_="if nsa")
            date_text = date_span.text.strip() if date_span else None
            
            # Convertir la date au format datetime
            article_date = None
            if date_text:
                try:
                    article_date = datetime.strptime(date_text, "%d/%m/%Y").strftime("%Y-%m-%d")
                except ValueError:
                    article_date = "Date invalide"
            
            articles.append({
                "title": title,
                "link": link,
                "date": article_date
            })
    
    return articles

# Exécuter la fonction et afficher les résultats
articles = scrape_first_page(url)
for article in articles:
    print(f"Titre: {article['title']}, Lien: {article['link']}, Date: {article['date']}")

In [8]:
articles

[]

In [1]:
import requests
from bs4 import BeautifulSoup

lien = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"
 # votre lien ici
result = requests.get(lien)
src = result.content
soup = BeautifulSoup(src, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="fr-fr" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# agenceecofin: http://ogp.me/ns/fb/agenceecofin#" xml:lang="fr-fr" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="Recherche article" name="title"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <base href="https://www.agenceecofin.com/a-la-une/recherche-article"/>
  <meta content="text/html; charset=utf-8" http-equiv="content-type">
   <meta content="Agence Ecofin, Télécom, Banque, Energie, Com, Média, Droit, Assurances, Gestion publique, Tourisme" name="keywords">
    <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
    <meta content="L’agence Ecofin est une agence d’information économique africaine, créée en décembre 2010, basée à Yaoundé et à Genève. Elle rassemble une équipe de journalistes et d’experts, spécialisés dans les principaux secteurs africains de l’éco

In [5]:
table = soup.find("table", class_="ts") 

In [6]:
table

<table cellpadding="0" cellspacing="0" class="ts" style="margin-bottom:10px;"><tbody><tr><td class="top news-tbn" valign="top"><div><a href="/actualites/2501-125238-gabon-la-production-industrielle-de-bois-en-hausse-de-4-au-3e-trimestre-2024" style="text-decoration:none"><div class="news-tbn-con"> <div class="dmk2articlesfilter_results_image">
<img align="left" height="60" src="https://www.agenceecofin.com/media/k2/items/cache/51f4fb95425b1e473afe5f85ba023a43_XS.jpg" style="margin: 10px;width: 76px;max-width: 80px;" width="80">
</img></div>
</div></a></div></td>
<td class="tsw" valign="top"><h3 class="r" style="margin-top:0px;line-height:0.7em !IMPORTANT;"><a href="/actualites/2501-125238-gabon-la-production-industrielle-de-bois-en-hausse-de-4-au-3e-trimestre-2024" style="color: #1122CC;cursor: pointer;text-decoration: underline;font-size:13px;font-weight:normal;">Gabon : la production industrielle de bois en hausse de 4% au 3e trimestre 2024</a></h3><div class="slp"><span class="news-

In [7]:
# Supposons que vous avez déjà récupéré la table dans une variable appelée `table`
rows = table.find_all("td", class_="top news-tbn")  # Récupérer les lignes des articles

# Liste pour stocker les articles
articles = []

# Boucle pour extraire les détails de chaque article
for row in rows:
    # Extraire le lien, le titre et la date
    link_tag = row.find("a")
    title = link_tag.text.strip() if link_tag else "Titre indisponible"
    relative_link = link_tag["href"] if link_tag else None
    link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None

    # Trouver la date dans le conteneur suivant
    date_span = row.find_next("span", class_="f nsa")
    date_text = date_span.text.strip() if date_span else None
    
    # Ajouter les détails à la liste des articles
    articles.append({
        "title": title,
        "link": link,
        "date": date_text
    })

# Afficher les résultats
if articles:
    for article in articles:
        print(f"Titre: {article['title']}, Lien: {article['link']}, Date: {article['date']}")
else:
    print("Aucun article trouvé.")

Titre: , Lien: https://www.agenceecofin.com/actualites/2501-125238-gabon-la-production-industrielle-de-bois-en-hausse-de-4-au-3e-trimestre-2024, Date: 25/01/2025


In [8]:
import requests
from bs4 import BeautifulSoup

# URL de la page principale
url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"

# Récupérer le contenu HTML de la page
try:
    response = requests.get(url)
    response.raise_for_status()  # Vérifie les erreurs de requête
    html = response.content
except requests.RequestException as e:
    print(f"Erreur lors de la récupération de {url}: {e}")
    exit()

# Utiliser BeautifulSoup pour analyser le HTML
soup = BeautifulSoup(html, "html.parser")

# Récupérer toutes les tables contenant des articles
tables = soup.find_all("table", class_="ts")  # Récupérer toutes les tables

# Liste pour stocker les articles
articles = []

# Boucle à travers chaque table
for table in tables:
    # Récupérer les lignes des articles
    rows = table.find_all("td", class_="top news-tbn")  # Récupérer les lignes des articles
    for row in rows:
        # Extraire le lien, le titre et la date
        link_tag = row.find("a")
        title = link_tag.text.strip() if link_tag else "Titre indisponible"
        relative_link = link_tag["href"] if link_tag else None
        link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None

        # Trouver la date dans le conteneur suivant
        date_span = row.find_next("span", class_="f nsa")
        date_text = date_span.text.strip() if date_span else None
        
        # Ajouter les détails à la liste des articles
        articles.append({
            "title": title,
            "link": link,
            "date": date_text
        })

# Afficher les résultats
if articles:
    for article in articles:
        print(f"Titre: {article['title']}, Lien: {article['link']}, Date: {article['date']}")
else:
    print("Aucun article trouvé.")

Titre: , Lien: https://www.agenceecofin.com/actualites/2501-125238-gabon-la-production-industrielle-de-bois-en-hausse-de-4-au-3e-trimestre-2024, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites-infrastructures/2501-125237-energies-trump-dramani-mahama-et-labonne-parmi-les-40-personnalites-a-suivre-en-2025, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites-culture/2501-125236-l-art-negre-connais-pas-l-hommage-incompris-de-picasso-a-l-art-africain, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites/2501-125235-gestion-publique-africaine-top-5-des-infos-marquantes-de-la-semaine, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites/2501-125234-le-fida-debloque-145-millions-pour-developpement-rural-a-madagascar, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites-services/2501-125233-plusieurs-essais-cliniques-prometteurs-en-afrique-le-point-sante-hebdomadaire, Date: 25/01/2025
Titre: , 

In [9]:
articles

[{'title': '',
  'link': 'https://www.agenceecofin.com/actualites/2501-125238-gabon-la-production-industrielle-de-bois-en-hausse-de-4-au-3e-trimestre-2024',
  'date': '25/01/2025'},
 {'title': '',
  'link': 'https://www.agenceecofin.com/actualites-infrastructures/2501-125237-energies-trump-dramani-mahama-et-labonne-parmi-les-40-personnalites-a-suivre-en-2025',
  'date': '25/01/2025'},
 {'title': '',
  'link': 'https://www.agenceecofin.com/actualites-culture/2501-125236-l-art-negre-connais-pas-l-hommage-incompris-de-picasso-a-l-art-africain',
  'date': '25/01/2025'},
 {'title': '',
  'link': 'https://www.agenceecofin.com/actualites/2501-125235-gestion-publique-africaine-top-5-des-infos-marquantes-de-la-semaine',
  'date': '25/01/2025'},
 {'title': '',
  'link': 'https://www.agenceecofin.com/actualites/2501-125234-le-fida-debloque-145-millions-pour-developpement-rural-a-madagascar',
  'date': '25/01/2025'},
 {'title': '',
  'link': 'https://www.agenceecofin.com/actualites-services/2501-1

In [10]:
import requests
from bs4 import BeautifulSoup

# URL de la page principale
url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"

# Récupérer le contenu HTML de la page
try:
    response = requests.get(url)
    response.raise_for_status()  # Vérifie les erreurs de requête
    html = response.content
except requests.RequestException as e:
    print(f"Erreur lors de la récupération de {url}: {e}")
    exit()

# Utiliser BeautifulSoup pour analyser le HTML
soup = BeautifulSoup(html, "html.parser")

# Récupérer toutes les tables contenant des articles
tables = soup.find_all("table", class_="ts")  # Récupérer toutes les tables

# Liste pour stocker les articles
articles = []

# Boucle à travers chaque table
for table in tables:
    # Récupérer les lignes des articles
    rows = table.find_all("td", class_="top news-tbn")
    for row in rows:
        # Extraire le lien, le titre et la date
        link_tag = row.find("a")
        title = link_tag.text.strip() if link_tag else "Titre indisponible"
        relative_link = link_tag["href"] if link_tag else None
        link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None

        # Trouver la date dans le conteneur suivant
        date_span = row.find_next("span", class_="f nsa")
        date_text = date_span.text.strip() if date_span else None
        
        # Récupérer le texte de l'article
        if link:
            try:
                article_response = requests.get(link)
                article_response.raise_for_status()
                article_html = article_response.content
                article_soup = BeautifulSoup(article_html, "html.parser")
                
                # Extraire le texte à partir de la structure fournie
                text_div = article_soup.find("div", class_="itemIntroText")
                article_text = ""
                if text_div:
                    paragraphs = text_div.find_all("p")
                    article_text = "\n".join(p.get_text(strip=True) for p in paragraphs)

            except requests.RequestException as e:
                print(f"Erreur lors de la récupération de l'article {link}: {e}")
                article_text = "Texte indisponible"
        else:
            article_text = "Lien indisponible"

        # Ajouter les détails à la liste des articles
        articles.append({
            "title": title,
            "link": link,
            "date": date_text,
            "text": article_text
        })

# Afficher les résultats
if articles:
    for article in articles:
        print(f"Titre: {article['title']}, Lien: {article['link']}, Date: {article['date']}, Texte: {article['text'][:100]}...")  # Affiche un extrait du texte
else:
    print("Aucun article trouvé.")

Titre: , Lien: https://www.agenceecofin.com/actualites/2501-125238-gabon-la-production-industrielle-de-bois-en-hausse-de-4-au-3e-trimestre-2024, Date: 25/01/2025, Texte: (Agence Ecofin) - Le Gabon a mené plusieurs actions ces dernières années pour valoriser la transform...
Titre: , Lien: https://www.agenceecofin.com/actualites-infrastructures/2501-125237-energies-trump-dramani-mahama-et-labonne-parmi-les-40-personnalites-a-suivre-en-2025, Date: 25/01/2025, Texte: (Agence Ecofin) - L'année 2025 s'annonce comme charnière pour le secteur énergétique africain. De no...
Titre: , Lien: https://www.agenceecofin.com/actualites-culture/2501-125236-l-art-negre-connais-pas-l-hommage-incompris-de-picasso-a-l-art-africain, Date: 25/01/2025, Texte: (Agence Ecofin) - Interrogé en 1920 par un critique sur son rapport aux arts venus d’Afrique, Pablo ...
Titre: , Lien: https://www.agenceecofin.com/actualites/2501-125235-gestion-publique-africaine-top-5-des-infos-marquantes-de-la-semaine, Date: 25/01/202

In [11]:
articles

[{'title': '',
  'link': 'https://www.agenceecofin.com/actualites/2501-125238-gabon-la-production-industrielle-de-bois-en-hausse-de-4-au-3e-trimestre-2024',
  'date': '25/01/2025',
  'text': '(Agence Ecofin) - Le Gabon a mené plusieurs actions ces dernières années pour valoriser la transformation industrielle de son bois, volet porteur d’une économie jusqu’ici dominée par le pétrole.\nL’indice composite des activités industrielles du domaine du bois au Gabon a grimpé de 4% en glissement trimestriel au 3etrimestre 2024, selon la dernière note conjoncturelle du ministère de l’Economie. Cette appréciation s’explique,relaieLe Nouveau Gabon, par la stabilité de la demande chinoise, le géant asiatique étant le principal client pour le bois gabonais, notamment celui de placage et le contreplaqué.\nL’indice de production du contreplaqué a ainsi augmenté de 19% par rapport au 2etrimestre 2024, poussé selon le ministère par la demande en Asie, en UE et sur le marché local. L’indice pour le bois 

In [12]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# URL de base de la pagination
base_url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"

# Date actuelle
current_date = datetime.now()

# Liste pour stocker les articles
articles = []

# Fonction pour extraire les articles d'une page donnée
def extract_articles(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        html = response.content
        soup = BeautifulSoup(html, "html.parser")

        # Récupérer toutes les tables contenant des articles
        tables = soup.find_all("table", class_="ts")

        for table in tables:
            rows = table.find_all("td", class_="top news-tbn")
            for row in rows:
                link_tag = row.find("a")
                title = link_tag.text.strip() if link_tag else "Titre indisponible"
                relative_link = link_tag["href"] if link_tag else None
                link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None

                date_span = row.find_next("span", class_="f nsa")
                date_text = date_span.text.strip() if date_span else None

                if date_text:
                    try:
                        article_date = datetime.strptime(date_text, "%d %B %Y")
                        if current_date - timedelta(days=7) <= article_date <= current_date:
                            articles.append({
                                "title": title,
                                "link": link,
                                "date": date_text
                            })
                    except ValueError:
                        print(f"Date invalide pour l'article: {title}")

    except requests.RequestException as e:
        print(f"Erreur lors de la récupération de {url}: {e}")

# Extraire les articles de la première page
extract_articles(base_url)

# Gérer la pagination
while True:
    # Chercher le lien vers la page suivante
    soup = BeautifulSoup(requests.get(base_url).content, "html.parser")
    pagination = soup.find("div", class_="pagination")
    
    if pagination:
        next_page = pagination.find("a", text="Suivant")  # Trouver le lien "Suivant"
        if next_page:
            next_page_url = next_page['href']
            if next_page_url.startswith('/'):
                next_page_url = f"https://www.agenceecofin.com{next_page_url}"
            extract_articles(next_page_url)
            base_url = next_page_url  # Mettre à jour l'URL de base pour la prochaine itération
        else:
            break  # Sortir si aucune page suivante n'est trouvée
    else:
        break  # Sortir si la pagination n'est pas trouvée

# Afficher les résultats
if articles:
    for article in articles:
        print(f"Titre: {article['title']}, Lien: {article['link']}, Date: {article['date']}")
else:
    print("Aucun article trouvé dans la semaine.")

Date invalide pour l'article: 
Date invalide pour l'article: 
Date invalide pour l'article: 
Date invalide pour l'article: 
Date invalide pour l'article: 
Date invalide pour l'article: 
Date invalide pour l'article: 
Aucun article trouvé dans la semaine.


In [13]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# URL de base de la pagination
base_url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"

# Date actuelle
current_date = datetime.now()

# Liste pour stocker les articles
articles = []

# Fonction pour extraire les articles d'une page donnée
def extract_articles(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        html = response.content
        soup = BeautifulSoup(html, "html.parser")

        # Récupérer toutes les tables contenant des articles
        tables = soup.find_all("table", class_="ts")

        for table in tables:
            rows = table.find_all("td", class_="top news-tbn")
            for row in rows:
                link_tag = row.find("a")
                title = link_tag.text.strip() if link_tag else "Titre indisponible"
                relative_link = link_tag["href"] if link_tag else None
                link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None

                date_span = row.find_next("span", class_="f nsa")
                date_text = date_span.text.strip() if date_span else None

                if date_text:
                    print(f"Date extraite: {date_text}")  # Impression pour débogage
                    try:
                        # Ajuster le format pour correspondre à '25/01/2025'
                        article_date = datetime.strptime(date_text, "%d/%m/%Y")
                        if current_date - timedelta(days=7) <= article_date <= current_date:
                            articles.append({
                                "title": title,
                                "link": link,
                                "date": date_text
                            })
                    except ValueError:
                        print(f"Date invalide pour l'article: {title}")

    except requests.RequestException as e:
        print(f"Erreur lors de la récupération de {url}: {e}")

# Extraire les articles de la première page
extract_articles(base_url)

# Gérer la pagination
while True:
    soup = BeautifulSoup(requests.get(base_url).content, "html.parser")
    pagination = soup.find("div", class_="pagination")
    
    if pagination:
        next_page = pagination.find("a", text="Suivant")
        if next_page:
            next_page_url = next_page['href']
            if next_page_url.startswith('/'):
                next_page_url = f"https://www.agenceecofin.com{next_page_url}"
            extract_articles(next_page_url)
            base_url = next_page_url
        else:
            break
    else:
        break

# Afficher les résultats
if articles:
    for article in articles:
        print(f"Titre: {article['title']}, Lien: {article['link']}, Date: {article['date']}")
else:
    print("Aucun article trouvé dans la semaine.")

Date extraite: 25/01/2025
Date extraite: 25/01/2025
Date extraite: 25/01/2025
Date extraite: 25/01/2025
Date extraite: 25/01/2025
Date extraite: 25/01/2025
Date extraite: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites/2501-125238-gabon-la-production-industrielle-de-bois-en-hausse-de-4-au-3e-trimestre-2024, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites-infrastructures/2501-125237-energies-trump-dramani-mahama-et-labonne-parmi-les-40-personnalites-a-suivre-en-2025, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites-culture/2501-125236-l-art-negre-connais-pas-l-hommage-incompris-de-picasso-a-l-art-africain, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites/2501-125235-gestion-publique-africaine-top-5-des-infos-marquantes-de-la-semaine, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites/2501-125234-le-fida-debloque-145-millions-pour-developpement-rural-a-madagascar, Date: 25/01/2025

In [14]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# URL de base de la pagination
base_url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"

# Date actuelle
current_date = datetime.now()

# Liste pour stocker les articles
articles = []

# Fonction pour extraire les articles d'une page donnée
def extract_articles(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        html = response.content
        soup = BeautifulSoup(html, "html.parser")

        # Récupérer toutes les tables contenant des articles
        tables = soup.find_all("table", class_="ts")

        for table in tables:
            rows = table.find_all("td", class_="top news-tbn")
            for row in rows:
                link_tag = row.find("a")
                title = link_tag.text.strip() if link_tag else "Titre indisponible"
                relative_link = link_tag["href"] if link_tag else None
                link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None

                date_span = row.find_next("span", class_="f nsa")
                date_text = date_span.text.strip() if date_span else None

                if date_text:
                    print(f"Date extraite: {date_text}")  # Impression pour débogage
                    try:
                        # Ajuster le format pour correspondre à '25/01/2025'
                        article_date = datetime.strptime(date_text, "%d/%m/%Y")
                        if current_date - timedelta(days=7) <= article_date <= current_date:
                            articles.append({
                                "title": title,
                                "link": link,
                                "date": date_text
                            })
                    except ValueError:
                        print(f"Date invalide pour l'article: {title}")

    except requests.RequestException as e:
        print(f"Erreur lors de la récupération de {url}: {e}")

# Extraire les articles de la première page
extract_articles(base_url)

# Gérer la pagination
while True:
    try:
        # Chercher le lien vers la page suivante
        response = requests.get(base_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        pagination = soup.find("div", class_="pagination")
        
        if pagination:
            next_page = pagination.find("a", text="Suivant")
            if next_page:
                next_page_url = next_page['href']
                if next_page_url.startswith('/'):
                    next_page_url = f"https://www.agenceecofin.com{next_page_url}"
                base_url = next_page_url  # Mettre à jour l'URL de base pour la prochaine itération
                extract_articles(base_url)  # Extraire des articles de la nouvelle page
            else:
                break  # Sortir si aucune page suivante n'est trouvée
        else:
            break  # Sortir si la pagination n'est pas trouvée

    except requests.RequestException as e:
        print(f"Erreur lors de la récupération de la page suivante: {e}")
        break

# Afficher les résultats
if articles:
    for article in articles:
        print(f"Titre: {article['title']}, Lien: {article['link']}, Date: {article['date']}")
else:
    print("Aucun article trouvé dans la semaine.")

Date extraite: 25/01/2025
Date extraite: 25/01/2025
Date extraite: 25/01/2025
Date extraite: 25/01/2025
Date extraite: 25/01/2025
Date extraite: 25/01/2025
Date extraite: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites/2501-125238-gabon-la-production-industrielle-de-bois-en-hausse-de-4-au-3e-trimestre-2024, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites-infrastructures/2501-125237-energies-trump-dramani-mahama-et-labonne-parmi-les-40-personnalites-a-suivre-en-2025, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites-culture/2501-125236-l-art-negre-connais-pas-l-hommage-incompris-de-picasso-a-l-art-africain, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites/2501-125235-gestion-publique-africaine-top-5-des-infos-marquantes-de-la-semaine, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites/2501-125234-le-fida-debloque-145-millions-pour-developpement-rural-a-madagascar, Date: 25/01/2025

In [17]:
import requests
from bs4 import BeautifulSoup

# Fonction pour extraire les articles d'une page donnée
def extract_articles(url):
    articles = []  # Liste pour stocker les articles
    try:
        response = requests.get(url)
        response.raise_for_status()  # Vérifie les erreurs de requête
        html = response.content
        soup = BeautifulSoup(html, "html.parser")

        # Récupérer toutes les tables contenant des articles
        tables = soup.find_all("table", class_="ts")
        
        # Boucle à travers chaque table
        for table in tables:
            rows = table.find_all("td", class_="top news-tbn")
            for row in rows:
                link_tag = row.find("a")
                title = link_tag.text.strip() if link_tag else "Titre indisponible"
                relative_link = link_tag["href"] if link_tag else None
                link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None

                date_span = row.find_next("span", class_="f nsa")
                date_text = date_span.text.strip() if date_span else None
                
                articles.append({
                    "title": title,
                    "link": link,
                    "date": date_text
                })

        # Trouver le lien vers la page suivante
        next_page_link = soup.find("a", title="Suivant")
        next_page_url = next_page_link["href"] if next_page_link else None
        if next_page_url and next_page_url.startswith('/'):
            next_page_url = f"https://www.agenceecofin.com{next_page_url}"

        return articles, next_page_url
    
    except requests.RequestException as e:
        print(f"Erreur lors de la récupération de {url}: {e}")
        return articles, None

# URL de la première page
url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"

# Extraire les articles de la première page
articles, next_page = extract_articles(url)

# Afficher les articles de la première page
print("Articles de la première page :")
if articles:
    for article in articles:
        print(f"Titre: {article['title']}, Lien: {article['link']}, Date: {article['date']}")
else:
    print("Aucun article trouvé.")

# Cliquer sur le lien "Suivant" pour aller à la page suivante
if next_page:
    print(f"\nAller à la page suivante : {next_page}")
    # Extraire les articles de la page suivante
    articles_next, _ = extract_articles(next_page)

    # Afficher les articles de la page suivante
    print("\nArticles de la page suivante :")
    if articles_next:
        for article in articles_next:
            print(f"Titre: {article['title']}, Lien: {article['link']}, Date: {article['date']}")
    else:
        print("Aucun article trouvé sur la page suivante.")
else:
    print("Aucune page suivante.")

Articles de la première page :
Titre: , Lien: https://www.agenceecofin.com/actualites/2501-125238-gabon-la-production-industrielle-de-bois-en-hausse-de-4-au-3e-trimestre-2024, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites-infrastructures/2501-125237-energies-trump-dramani-mahama-et-labonne-parmi-les-40-personnalites-a-suivre-en-2025, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites-culture/2501-125236-l-art-negre-connais-pas-l-hommage-incompris-de-picasso-a-l-art-africain, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites/2501-125235-gestion-publique-africaine-top-5-des-infos-marquantes-de-la-semaine, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites/2501-125234-le-fida-debloque-145-millions-pour-developpement-rural-a-madagascar, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites-services/2501-125233-plusieurs-essais-cliniques-prometteurs-en-afrique-le-point-sante-hebdomada

In [21]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# Fonction pour extraire les articles d'une page donnée
def extract_articles(url):
    articles = []  # Liste pour stocker les articles
    try:
        response = requests.get(url)
        response.raise_for_status()  # Vérifie les erreurs de requête
        html = response.content
        soup = BeautifulSoup(html, "html.parser")

        # Récupérer toutes les tables contenant des articles
        tables = soup.find_all("table", class_="ts")
        
        # Boucle à travers chaque table
        for table in tables:
            rows = table.find_all("td", class_="top news-tbn")
            for row in rows:
                link_tag = row.find("a")
                title = link_tag.text.strip() if link_tag else "Titre indisponible"
                relative_link = link_tag["href"] if link_tag else None
                link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None

                date_span = row.find_next("span", class_="f nsa")
                date_text = date_span.text.strip() if date_span else None
                
                # Convertir la date en objet datetime
                if date_text:
                    article_date = datetime.strptime(date_text, "%d %m %Y")
                    articles.append({
                        "title": title,
                        "link": link,
                        "date": article_date
                    })

        # Trouver le lien vers la page suivante
        next_page_link = soup.find("a", title="Suivant")
        next_page_url = next_page_link["href"] if next_page_link else None
        if next_page_url and next_page_url.startswith('/'):
            next_page_url = f"https://www.agenceecofin.com{next_page_url}"

        return articles, next_page_url
    
    except requests.RequestException as e:
        print(f"Erreur lors de la récupération de {url}: {e}")
        return articles, None

# URL de la première page
url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"
all_articles = []
one_week_ago = datetime.now() - timedelta(days=7)

while True:
    print(f"Extraction des articles à partir de : {url}")
    articles, next_page = extract_articles(url)

    # Ajouter les articles extraits à la liste totale
    if articles:
        all_articles.extend(articles)

        # Vérifier si la date de l'article le plus récent est supérieure à une semaine
        if articles[-1]["date"] > one_week_ago:
            print("Un article a été trouvé au-delà d'une semaine, arrêt de l'extraction.")
            break

    # Aller à la page suivante si disponible
    if next_page:
        url = next_page
    else:
        break

# Afficher tous les articles récupérés
print("\nTous les articles extraits :")
if all_articles:
    for article in all_articles:
        print(f"Titre: {article['title']}, Lien: {article['link']}, Date: {article['date'].strftime('%d %m %Y')}")
else:
    print("Aucun article trouvé.")

Extraction des articles à partir de : https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results


ValueError: time data '25/01/2025' does not match format '%d %m %Y'

In [22]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# Fonction pour extraire les articles d'une page donnée
def extract_articles(url):
    articles = []  # Liste pour stocker les articles
    try:
        response = requests.get(url)
        response.raise_for_status()  # Vérifie les erreurs de requête
        html = response.content
        soup = BeautifulSoup(html, "html.parser")

        # Récupérer toutes les tables contenant des articles
        tables = soup.find_all("table", class_="ts")
        
        # Boucle à travers chaque table
        for table in tables:
            rows = table.find_all("td", class_="top news-tbn")
            for row in rows:
                link_tag = row.find("a")
                title = link_tag.text.strip() if link_tag else "Titre indisponible"
                relative_link = link_tag["href"] if link_tag else None
                link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None

                date_span = row.find_next("span", class_="f nsa")
                date_text = date_span.text.strip() if date_span else None
                
                # Convertir la date en objet datetime
                if date_text:
                    # Changer le format de date ici
                    article_date = datetime.strptime(date_text, "%d/%m/%Y")
                    articles.append({
                        "title": title,
                        "link": link,
                        "date": article_date
                    })

        # Trouver le lien vers la page suivante
        next_page_link = soup.find("a", title="Suivant")
        next_page_url = next_page_link["href"] if next_page_link else None
        if next_page_url and next_page_url.startswith('/'):
            next_page_url = f"https://www.agenceecofin.com{next_page_url}"

        return articles, next_page_url
    
    except requests.RequestException as e:
        print(f"Erreur lors de la récupération de {url}: {e}")
        return articles, None

# URL de la première page
url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"
all_articles = []
one_week_ago = datetime.now() - timedelta(days=7)

while True:
    print(f"Extraction des articles à partir de : {url}")
    articles, next_page = extract_articles(url)

    # Ajouter les articles extraits à la liste totale
    if articles:
        all_articles.extend(articles)

        # Vérifier si la date de l'article le plus récent est supérieure à une semaine
        if articles[-1]["date"] > one_week_ago:
            print("Un article a été trouvé au-delà d'une semaine, arrêt de l'extraction.")
            break

    # Aller à la page suivante si disponible
    if next_page:
        url = next_page
    else:
        break

# Afficher tous les articles récupérés
print("\nTous les articles extraits :")
if all_articles:
    for article in all_articles:
        print(f"Titre: {article['title']}, Lien: {article['link']}, Date: {article['date'].strftime('%d/%m/%Y')}")
else:
    print("Aucun article trouvé.")

Extraction des articles à partir de : https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results
Un article a été trouvé au-delà d'une semaine, arrêt de l'extraction.

Tous les articles extraits :
Titre: , Lien: https://www.agenceecofin.com/actualites/2501-125238-gabon-la-production-industrielle-de-bois-en-hausse-de-4-au-3e-trimestre-2024, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites-infrastructures/2501-125237-energies-trump-dramani-mahama-et-labonne-parmi-les-40-personnalites-a-suivre-en-2025, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actualites-culture/2501-125236-l-art-negre-connais-pas-l-hommage-incompris-de-picasso-a-l-art-africain, Date: 25/01/2025
Titre: , Lien: https://www.agenceecofin.com/actuali

In [23]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# Fonction pour extraire les articles d'une page donnée
def extract_articles(url):
    articles = []  # Liste pour stocker les articles
    try:
        response = requests.get(url)
        response.raise_for_status()  # Vérifie les erreurs de requête
        html = response.content
        soup = BeautifulSoup(html, "html.parser")

        # Récupérer toutes les tables contenant des articles
        tables = soup.find_all("table", class_="ts")
        
        # Boucle à travers chaque table
        for table in tables:
            rows = table.find_all("td", class_="top news-tbn")
            for row in rows:
                link_tag = row.find("a")
                title = link_tag.text.strip() if link_tag else "Titre indisponible"
                relative_link = link_tag["href"] if link_tag else None
                link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None

                date_span = row.find_next("span", class_="f nsa")
                date_text = date_span.text.strip() if date_span else None
                
                # Convertir la date en objet datetime
                if date_text:
                    article_date = datetime.strptime(date_text, "%d/%m/%Y")
                    articles.append({
                        "title": title,
                        "link": link,
                        "date": article_date
                    })

        # Trouver le lien vers la page suivante
        next_page_link = soup.find("a", title="Suivant")
        next_page_url = next_page_link["href"] if next_page_link else None
        if next_page_url and next_page_url.startswith('/'):
            next_page_url = f"https://www.agenceecofin.com{next_page_url}"

        return articles, next_page_url
    
    except requests.RequestException as e:
        print(f"Erreur lors de la récupération de {url}: {e}")
        return articles, None

# URL de la première page
url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"
all_articles = []
stop_date = datetime(2025, 1, 18)  # Date d'arrêt pour les articles

while True:
    print(f"Extraction des articles à partir de : {url}")
    articles, next_page = extract_articles(url)

    # Ajouter les articles extraits à la liste totale
    if articles:
        all_articles.extend(articles)

        # Vérifier si la date de l'article le plus récent est inférieure à 18/01/2025
        if articles[-1]["date"] < stop_date:
            print("Un article a été trouvé avant le 18/01/2025, arrêt de l'extraction.")
            break

    # Aller à la page suivante si disponible
    if next_page:
        url = next_page
    else:
        break

# Afficher tous les articles récupérés
print("\nTous les articles extraits :")
if all_articles:
    for article in all_articles:
        print(f"Titre: {article['title']}, Lien: {article['link']}, Date: {article['date'].strftime('%d/%m/%Y')}")
else:
    print("Aucun article trouvé.")

Extraction des articles à partir de : https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results
Extraction des articles à partir de : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=7
Extraction des articles à partir de : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=14
Extraction des articles à partir de : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSe

In [24]:
articles

[{'title': '',
  'link': 'https://www.agenceecofin.com/actualites-culture/1801-125021-maroc-la-foire-d-art-contemporain-africain-1-54-se-tiendra-du-30-janvier-au-2-fevrier',
  'date': datetime.datetime(2025, 1, 18, 0, 0)},
 {'title': '',
  'link': 'https://www.agenceecofin.com/actualites-services/1801-125020-marburg-en-tanzanie-penurie-de-vaccins-bcg-au-kenya-le-point-sante-hebdomadaire',
  'date': datetime.datetime(2025, 1, 18, 0, 0)},
 {'title': '',
  'link': 'https://www.agenceecofin.com/actualites-industries/1801-125019-zambie-serengeti-energy-et-western-solar-power-actent-le-projet-de-centrale-pv-dilute-32-mwp',
  'date': datetime.datetime(2025, 1, 18, 0, 0)},
 {'title': '',
  'link': 'https://www.agenceecofin.com/actualites/1801-125018-sur-les-questions-de-genre-le-vodun-est-la-religion-la-plus-evoluee-mambo-zila-moyo-dignitaire-vodun',
  'date': datetime.datetime(2025, 1, 18, 0, 0)},
 {'title': '',
  'link': 'https://www.agenceecofin.com/actualites-services/1701-125017-maroc-lon

In [33]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import csv

# Fonction pour extraire les articles d'une page donnée
def extract_articles(url):
    articles = []  # Liste pour stocker les articles
    try:
        response = requests.get(url)
        response.raise_for_status()  # Vérifie les erreurs de requête
        html = response.content
        soup = BeautifulSoup(html, "html.parser")

        # Récupérer toutes les tables contenant des articles
        tables = soup.find_all("table", class_="ts")
        
        # Boucle à travers chaque table
        for table in tables:
            rows = table.find_all("td", class_="top news-tbn")
            for row in rows:
                link_tag = row.find("a")
                title = link_tag.text.strip() if link_tag else "Titre indisponible"
                relative_link = link_tag["href"] if link_tag else None
                link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None

                date_span = row.find_next("span", class_="f nsa")
                date_text = date_span.text.strip() if date_span else None
                
                # Convertir la date en objet datetime
                if date_text:
                    article_date = datetime.strptime(date_text, "%d/%m/%Y")
                    articles.append({
                        "title": title,
                        "link": link,
                        "date": article_date
                    })

        # Trouver le lien vers la page suivante
        next_page_link = soup.find("a", title="Suivant")
        next_page_url = next_page_link["href"] if next_page_link else None
        if next_page_url and next_page_url.startswith('/'):
            next_page_url = f"https://www.agenceecofin.com{next_page_url}"

        return articles, next_page_url
    
    except requests.RequestException as e:
        print(f"Erreur lors de la récupération de {url}: {e}")
        return articles, None

# URL de la première page
url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"
all_articles = []
stop_date = datetime(2025, 1, 18)  # Date d'arrêt pour les articles

while True:
    print(f"Extraction des articles à partir de : {url}")
    articles, next_page = extract_articles(url)

    # Ajouter les articles extraits à la liste totale
    if articles:
        all_articles.extend(articles)

        # Vérifier si la date de l'article le plus récent est inférieure à 18/01/2025
        if articles[-1]["date"] < stop_date:
            print("Un article a été trouvé avant le 18/01/2025, arrêt de l'extraction.")
            break

    # Aller à la page suivante si disponible
    if next_page:
        url = next_page
    else:
        break

# Afficher tous les articles récupérés
print("\nTous les articles extraits :")
if all_articles:
    for article in all_articles:
        print(f"Titre: {article['title']}, Lien: {article['link']}, Date: {article['date'].strftime('%d/%m/%Y')}")
else:
    print("Aucun article trouvé.")

# Enregistrer les articles dans un fichier CSV
with open("articles.csv", mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Titre", "Lien", "Date"])  # Écrire l'en-tête
    for article in all_articles:
        writer.writerow([article['title'], article['link'], article['date'].strftime('%d/%m/%Y')])

print("\nTous les articles extraits et enregistrés dans 'articles.csv'.")

Extraction des articles à partir de : https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results
Extraction des articles à partir de : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=7
Extraction des articles à partir de : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=14
Extraction des articles à partir de : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSe

In [34]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import csv

# URL de la page principale
url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"

# Récupérer le contenu HTML de la page
try:
    response = requests.get(url)
    response.raise_for_status()  # Vérifie les erreurs de requête
    html = response.content
except requests.RequestException as e:
    print(f"Erreur lors de la récupération de {url}: {e}")
    exit()

# Utiliser BeautifulSoup pour analyser le HTML
soup = BeautifulSoup(html, "html.parser")

# Récupérer toutes les tables contenant des articles
tables = soup.find_all("table", class_="ts")  # Récupérer toutes les tables

# Liste pour stocker les articles
all_articles = []
stop_date = datetime.now() - timedelta(days=7)  # Date d'arrêt dynamique pour 7 jours en arrière

# Boucle à travers chaque table
for table in tables:
    # Récupérer les lignes des articles
    rows = table.find_all("td", class_="top news-tbn")
    for row in rows:
        # Extraire le lien, le titre et la date
        link_tag = row.find("a")
        title = link_tag.text.strip() if link_tag else "Titre indisponible"
        relative_link = link_tag["href"] if link_tag else None
        link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None

        # Trouver la date dans le conteneur suivant
        date_span = row.find_next("span", class_="f nsa")
        date_text = date_span.text.strip() if date_span else None
        
        # Convertir la date en objet datetime
        if date_text:
            article_date = datetime.strptime(date_text, "%d/%m/%Y")
            # Vérifier si la date est plus ancienne que la date limite
            if article_date < stop_date:
                print("Arrêt de l'extraction, un article est plus ancien que la date limite.")
                break

            # Récupérer le texte de l'article
            if link:
                try:
                    article_response = requests.get(link)
                    article_response.raise_for_status()
                    article_html = article_response.content
                    article_soup = BeautifulSoup(article_html, "html.parser")
                    
                    # Extraire le texte à partir de la structure fournie
                    text_div = article_soup.find("div", class_="itemIntroText")
                    article_text = ""
                    if text_div:
                        paragraphs = text_div.find_all("p")
                        article_text = "\n".join(p.get_text(strip=True) for p in paragraphs)

                except requests.RequestException as e:
                    print(f"Erreur lors de la récupération de l'article {link}: {e}")
                    article_text = "Texte indisponible"
            else:
                article_text = "Lien indisponible"

            # Ajouter les détails à la liste des articles
            all_articles.append({
                "title": title,
                "link": link,
                "date": date_text,
                "text": article_text
            })
        else:
            print("Date indisponible pour l'article.")

# Afficher les résultats
if all_articles:
    for article in all_articles:
        print(f"Titre: {article['title']}, Lien: {article['link']}, Date: {article['date']}, Texte: {article['text'][:100]}...")  # Affiche un extrait du texte
else:
    print("Aucun article trouvé.")

# Enregistrer les articles dans un fichier CSV
with open("articles.csv", mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Titre", "Lien", "Date", "Texte"])  # Écrire l'en-tête
    for article in all_articles:
        writer.writerow([article['title'], article['link'], article['date'], article['text']])

print("\nTous les articles extraits et enregistrés dans 'articles.csv'.")

Titre: , Lien: https://www.agenceecofin.com/actualites/2501-125238-gabon-la-production-industrielle-de-bois-en-hausse-de-4-au-3e-trimestre-2024, Date: 25/01/2025, Texte: (Agence Ecofin) - Le Gabon a mené plusieurs actions ces dernières années pour valoriser la transform...
Titre: , Lien: https://www.agenceecofin.com/actualites-infrastructures/2501-125237-energies-trump-dramani-mahama-et-labonne-parmi-les-40-personnalites-a-suivre-en-2025, Date: 25/01/2025, Texte: (Agence Ecofin) - L'année 2025 s'annonce comme charnière pour le secteur énergétique africain. De no...
Titre: , Lien: https://www.agenceecofin.com/actualites-culture/2501-125236-l-art-negre-connais-pas-l-hommage-incompris-de-picasso-a-l-art-africain, Date: 25/01/2025, Texte: (Agence Ecofin) - Interrogé en 1920 par un critique sur son rapport aux arts venus d’Afrique, Pablo ...
Titre: , Lien: https://www.agenceecofin.com/actualites/2501-125235-gestion-publique-africaine-top-5-des-infos-marquantes-de-la-semaine, Date: 25/01/202

In [35]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import csv

# Fonction pour extraire les articles d'une page donnée
def extract_articles(url):
    articles = []  # Liste pour stocker les articles
    try:
        response = requests.get(url)
        response.raise_for_status()  # Vérifie les erreurs de requête
        html = response.content
        soup = BeautifulSoup(html, "html.parser")

        # Récupérer toutes les tables contenant des articles
        tables = soup.find_all("table", class_="ts")
        
        # Boucle à travers chaque table
        for table in tables:
            # Récupérer les lignes des articles
            rows = table.find_all("td", class_="top news-tbn")
            for row in rows:
                # Extraire le lien, le titre et la date
                link_tag = row.find("a")
                title = link_tag.text.strip() if link_tag else "Titre indisponible"
                relative_link = link_tag["href"] if link_tag else None
                link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None

                # Trouver la date dans le conteneur suivant
                date_span = row.find_next("span", class_="f nsa")
                date_text = date_span.text.strip() if date_span else None

                # Récupérer le texte de l'article
                article_text = ""
                if link:
                    try:
                        article_response = requests.get(link)
                        article_response.raise_for_status()
                        article_html = article_response.content
                        article_soup = BeautifulSoup(article_html, "html.parser")
                        
                        # Extraire le texte à partir de la structure fournie
                        text_div = article_soup.find("div", class_="itemIntroText")
                        if text_div:
                            paragraphs = text_div.find_all("p")
                            article_text = "\n".join(p.get_text(strip=True) for p in paragraphs)

                    except requests.RequestException as e:
                        print(f"Erreur lors de la récupération de l'article {link}: {e}")
                        article_text = "Texte indisponible"
                else:
                    article_text = "Lien indisponible"

                # Ajouter les détails de l'article à la liste
                articles.append({
                    "title": title,
                    "link": link,
                    "date": date_text,
                    "text": article_text
                })

        # Trouver le lien vers la page suivante
        next_page_link = soup.find("a", title="Suivant")
        next_page_url = next_page_link["href"] if next_page_link else None
        if next_page_url and next_page_url.startswith('/'):
            next_page_url = f"https://www.agenceecofin.com{next_page_url}"

        return articles, next_page_url
    
    except requests.RequestException as e:
        print(f"Erreur lors de la récupération de {url}: {e}")
        return articles, None

# URL de la première page
url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"
all_articles = []
stop_date = datetime.now() - timedelta(days=7)  # Date d'arrêt dynamique pour 7 jours en arrière

while True:
    print(f"Extraction des articles à partir de : {url}")
    articles, next_page = extract_articles(url)

    # Ajouter les articles extraits à la liste totale
    if articles:
        all_articles.extend(articles)

        # Vérifier si la date de l'article le plus récent est inférieure à stop_date
        if articles[-1]["date"] < stop_date:
            print("Un article a été trouvé avant la date limite de 7 jours, arrêt de l'extraction.")
            break

    # Aller à la page suivante si disponible
    if next_page:
        url = next_page
    else:
        break

# Afficher tous les articles récupérés
print("\nTous les articles extraits :")
if all_articles:
    for article in all_articles:
        print(f"Titre: {article['title']}, Lien: {article['link']}, Date: {article['date']}, Texte: {article['text'][:100]}...")  # Affiche les 100 premiers caractères du texte
else:
    print("Aucun article trouvé.")

# Enregistrer les articles dans un fichier CSV
with open("articles.csv", mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Titre", "Lien", "Date", "Texte"])  # Écrire l'en-tête
    for article in all_articles:
        writer.writerow([article['title'], article['link'], article['date'], article['text']])

print("\nTous les articles extraits et enregistrés dans 'articles.csv'.")

Extraction des articles à partir de : https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results


TypeError: '<' not supported between instances of 'str' and 'datetime.datetime'

In [36]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import csv

# Fonction pour extraire les articles d'une page donnée
def extract_articles(url):
    articles = []  # Liste pour stocker les articles
    try:
        response = requests.get(url)
        response.raise_for_status()  # Vérifie les erreurs de requête
        html = response.content
        soup = BeautifulSoup(html, "html.parser")

        # Récupérer toutes les tables contenant des articles
        tables = soup.find_all("table", class_="ts")
        
        # Boucle à travers chaque table
        for table in tables:
            # Récupérer les lignes des articles
            rows = table.find_all("td", class_="top news-tbn")
            for row in rows:
                # Extraire le lien, le titre et la date
                link_tag = row.find("a")
                title = link_tag.text.strip() if link_tag else "Titre indisponible"
                relative_link = link_tag["href"] if link_tag else None
                link = f"https://www.agenceecofin.com{relative_link}" if relative_link else None

                # Trouver la date dans le conteneur suivant
                date_span = row.find_next("span", class_="f nsa")
                date_text = date_span.text.strip() if date_span else None

                # Convertir la date en objet datetime
                article_date = None
                if date_text:
                    try:
                        article_date = datetime.strptime(date_text, "%d/%m/%Y")
                    except ValueError:
                        print(f"Erreur de format de date pour l'article: {title}")
                        article_date = None

                # Récupérer le texte de l'article
                article_text = ""
                if link:
                    try:
                        article_response = requests.get(link)
                        article_response.raise_for_status()
                        article_html = article_response.content
                        article_soup = BeautifulSoup(article_html, "html.parser")
                        
                        # Extraire le texte à partir de la structure fournie
                        text_div = article_soup.find("div", class_="itemIntroText")
                        if text_div:
                            paragraphs = text_div.find_all("p")
                            article_text = "\n".join(p.get_text(strip=True) for p in paragraphs)

                    except requests.RequestException as e:
                        print(f"Erreur lors de la récupération de l'article {link}: {e}")
                        article_text = "Texte indisponible"
                else:
                    article_text = "Lien indisponible"

                # Ajouter les détails de l'article à la liste
                articles.append({
                    "title": title,
                    "link": link,
                    "date": article_date,
                    "text": article_text
                })

        # Trouver le lien vers la page suivante
        next_page_link = soup.find("a", title="Suivant")
        next_page_url = next_page_link["href"] if next_page_link else None
        if next_page_url and next_page_url.startswith('/'):
            next_page_url = f"https://www.agenceecofin.com{next_page_url}"

        return articles, next_page_url
    
    except requests.RequestException as e:
        print(f"Erreur lors de la récupération de {url}: {e}")
        return articles, None

# URL de la première page
url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"
all_articles = []
stop_date = datetime.now() - timedelta(days=7)  # Date d'arrêt dynamique pour 7 jours en arrière

while True:
    print(f"Extraction des articles à partir de : {url}")
    articles, next_page = extract_articles(url)

    # Ajouter les articles extraits à la liste totale
    if articles:
        all_articles.extend(articles)

        # Vérifier si la date de l'article le plus récent est inférieure à stop_date
        # On vérifie que article_date n'est pas None
        if articles[-1]["date"] and articles[-1]["date"] < stop_date:
            print("Un article a été trouvé avant la date limite de 7 jours, arrêt de l'extraction.")
            break

    # Aller à la page suivante si disponible
    if next_page:
        url = next_page
    else:
        break

# Afficher tous les articles récupérés
print("\nTous les articles extraits :")
if all_articles:
    for article in all_articles:
        print(f"Titre: {article['title']}, Lien: {article['link']}, Date: {article['date']}, Texte: {article['text'][:100]}...")  # Affiche les 100 premiers caractères du texte
else:
    print("Aucun article trouvé.")

# Enregistrer les articles dans un fichier CSV
with open("articles.csv", mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Titre", "Lien", "Date", "Texte"])  # Écrire l'en-tête
    for article in all_articles:
        writer.writerow([article['title'], article['link'], article['date'].strftime('%d/%m/%Y') if article['date'] else '', article['text']])

print("\nTous les articles extraits et enregistrés dans 'articles.csv'.")

Extraction des articles à partir de : https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results
Extraction des articles à partir de : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=7
Extraction des articles à partir de : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSearch=1&testlimitstart=14
Extraction des articles à partir de : https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=0&submit_y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterFrench=French&userSe